# Introduction to RAG (Retrieval Augmented Generation) with Structured Databases for Supply Chain Management

This notebook provides a beginner-friendly introduction to building a RAG (Retrieval Augmented Generation) system that queries structured databases instead of documents. We'll focus on supply chain management (SCM) in the context of geopolitical fragmentation.

## What is Database RAG?

Traditional RAG systems work with unstructured text documents:
1. Documents are split into chunks
2. Chunks are embedded into vectors
3. User questions are matched to relevant chunks using vector similarity
4. Retrieved chunks are sent to an LLM along with the question to generate an answer

Database RAG works differently:
1. User questions are translated into SQL queries
2. SQL queries are executed against a database
3. Query results are formatted as context
4. Context is sent to an LLM along with the question to generate an answer

This approach has several advantages:
- More precise retrieval through structured queries
- Better handling of numerical data and aggregations
- Ability to answer questions that require calculations
- More efficient for large datasets

## Supply Chain Management and Geopolitical Fragmentation

In today's world, supply chains are increasingly affected by geopolitical fragmentation:
- Tariff barriers pushing toward local supply chains
- Pandemic disruptions affecting global logistics
- Regional conflicts creating supply uncertainties
- Trade policies favoring domestic production

Our RAG system will help answer critical questions for supply chain managers:
- Which products are sourced locally vs globally?
- Which suppliers have the highest geopolitical risk?
- What alternative suppliers exist in different regions?
- How do tariff rates affect our procurement strategy?

## What We'll Build

In this notebook, we'll build a complete database RAG system that:
1. Creates a sample SQLite database with regions, suppliers, products, and sales data
2. Translates natural language questions into SQL queries using both rule-based and LLM approaches
3. Executes these queries and formats the results
4. Generates answers using various methods (simple extractive or LLM-based)
5. Provides an interactive UI for testing

## Setup

First, let's install the necessary dependencies:

In [27]:
# Install required packages
!pip install sqlite3 pandas numpy requests ipywidgets

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [28]:
# Import necessary libraries
import os
import re
import sqlite3
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, timedelta
import random

# Check if we're running in a Jupyter notebook
try:
    from IPython.display import display, HTML, clear_output
    import ipywidgets as widgets
    in_notebook = True
except ImportError:
    in_notebook = False
    print("Not running in a Jupyter notebook. Some interactive features may not work.")

# Database Setup

Let's start by creating a simple SQLite database with tables for suppliers, products, and sales. This will be our structured data source.

In [29]:
# Function to create the database schema
def setup_database(db_path):
    """Create a new SQLite database with our schema

    Args:
        db_path (str): Path to the SQLite database file

    Returns:
        sqlite3.Connection: Connection to the database
    """
    # Connect to the database (creates it if it doesn't exist)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Create regions table (new for geopolitical risk analysis)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS regions (
        region_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        risk_factor INTEGER NOT NULL
    )
    ''')

    # Create suppliers table (updated with region and tariff information)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS suppliers (
        supplier_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        country TEXT NOT NULL,
        region_id INTEGER NOT NULL,
        tariff_rate REAL NOT NULL,
        contact_email TEXT,
        FOREIGN KEY (region_id) REFERENCES regions (region_id)
    )
    ''')

    # Create products table (updated with is_purchased flag)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        category TEXT NOT NULL,
        price REAL NOT NULL,
        supplier_id INTEGER,
        is_purchased INTEGER NOT NULL,
        FOREIGN KEY (supplier_id) REFERENCES suppliers (supplier_id)
    )
    ''')

    # Create sales table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales (
        sale_id INTEGER PRIMARY KEY,
        product_id INTEGER,
        quantity INTEGER NOT NULL,
        sale_date TEXT NOT NULL,
        customer_name TEXT,
        total_amount REAL NOT NULL,
        FOREIGN KEY (product_id) REFERENCES products (product_id)
    )
    ''')

    # Commit the changes
    conn.commit()

    return conn

In [30]:
# Function to populate the database with sample data
def populate_sample_data(conn):
    """Add sample data to our database

    Args:
        conn: SQLite connection
    """
    cursor = conn.cursor()

    # Sample regions data with geopolitical risk factors (1-10 scale, 10 being highest risk)
    regions = [
        (1, "North America", 2),
        (2, "East Asia", 5),
        (3, "Europe", 3),
        (4, "Southeast Asia", 6),
        (5, "South America", 4),
        (6, "Middle East", 8)
    ]

    # Sample supplier data with region_id and tariff_rate
    suppliers = [
        (1, "Quantum Microchips", "USA", 1, 0.0, "contact@quantummicro.com"),
        (2, "Global Electronics", "China", 2, 25.0, "sales@globalelec.com"),
        (3, "European Components", "Germany", 3, 5.0, "info@eurocomp.eu"),
        (4, "Pacific Devices", "Japan", 2, 10.0, "support@pacificdev.jp"),
        (5, "Nordic Solutions", "Sweden", 3, 5.0, "hello@nordicsol.se"),
        (6, "Mexican Assemblies", "Mexico", 1, 0.0, "info@mexassemb.mx"),
        (7, "Vietnam Tech", "Vietnam", 4, 15.0, "sales@viettech.vn"),
        (8, "Brazilian Parts", "Brazil", 5, 20.0, "contact@brazparts.br"),
        (9, "UAE Semiconductors", "UAE", 6, 12.0, "info@uaesemi.ae")
    ]

    # Sample product data with is_purchased flag (1 = we purchase it, 0 = potential product)
    products = [
        (101, "High-Performance CPU", "Electronics", 299.99, 1, 1),
        (102, "Graphics Card Pro", "Electronics", 499.99, 1, 1),
        (103, "Memory Module 16GB", "Electronics", 89.99, 2, 1),
        (104, "SSD 1TB", "Storage", 129.99, 2, 1),
        (105, "HDD 4TB", "Storage", 99.99, 3, 1),
        (106, "Wireless Mouse", "Peripherals", 24.99, 3, 1),
        (107, "Mechanical Keyboard", "Peripherals", 79.99, 4, 1),
        (108, "27-inch Monitor", "Displays", 249.99, 4, 1),
        (109, "Wireless Headphones", "Audio", 149.99, 5, 1),
        (110, "Bluetooth Speaker", "Audio", 89.99, 5, 1),
        (111, "Laptop Pro", "Electronics", 1299.99, 1, 1),
        (112, "Tablet 10-inch", "Electronics", 399.99, 2, 1),
        (113, "Smartphone X", "Electronics", 699.99, 3, 0),
        (114, "Smartwatch", "Wearables", 199.99, 4, 0),
        (115, "Fitness Tracker", "Wearables", 99.99, 5, 0),
        (116, "Server Rack", "Infrastructure", 899.99, 6, 1),
        (117, "Network Switch", "Infrastructure", 349.99, 6, 1),
        (118, "Power Supply Unit", "Components", 89.99, 7, 1),
        (119, "Cooling System", "Components", 129.99, 7, 0),
        (120, "Optical Drive", "Storage", 59.99, 8, 0),
        (121, "Touch Display", "Displays", 399.99, 8, 0),
        (122, "Quantum Processor", "Advanced Electronics", 1499.99, 9, 0),
        (123, "AI Accelerator", "Advanced Electronics", 2499.99, 9, 0)
    ]

    # Generate sample sales data (only for products we purchase, is_purchased=1)
    sales = []
    sale_id = 1
    customers = ["TechCorp", "Innovate Inc", "Future Systems", "Smart Solutions", "Tech Retail", "Digital Experts", "Computing Giants"]

    # Get list of purchased product IDs
    purchased_product_ids = [p[0] for p in products if p[5] == 1]

    # Generate sales for Q1 2025 (January to March)
    start_date = datetime(2025, 1, 1)
    end_date = datetime(2025, 3, 31)
    current_date = start_date

    while current_date <= end_date:
        # Generate 1-5 sales per day
        for _ in range(random.randint(1, 5)):
            product_id = random.choice(purchased_product_ids)
            # Find the price of the selected product
            product_price = next(p[3] for p in products if p[0] == product_id)

            quantity = random.randint(1, 10)
            customer = random.choice(customers)
            total_amount = round(product_price * quantity, 2)

            sales.append((sale_id, product_id, quantity, current_date.strftime('%Y-%m-%d'), customer, total_amount))
            sale_id += 1

        current_date += timedelta(days=1)

    # Insert data into tables
    cursor.executemany('INSERT OR REPLACE INTO regions VALUES (?, ?, ?)', regions)
    cursor.executemany('INSERT OR REPLACE INTO suppliers VALUES (?, ?, ?, ?, ?, ?)', suppliers)
    cursor.executemany('INSERT OR REPLACE INTO products VALUES (?, ?, ?, ?, ?, ?)', products)
    cursor.executemany('INSERT OR REPLACE INTO sales VALUES (?, ?, ?, ?, ?, ?)', sales)

    # Commit the changes
    conn.commit()

    print(f"Database populated with {len(regions)} regions, {len(suppliers)} suppliers, {len(products)} products, and {len(sales)} sales records.")

In [31]:
# Create and populate our database
db_path = "supply_chain.db"
conn = setup_database(db_path)
populate_sample_data(conn)
print(f"Database created at {db_path}")

Database populated with 6 regions, 9 suppliers, 23 products, and 257 sales records.
Database created at supply_chain.db


# Database Exploration

Let's create a function to explore our database tables and their contents.

In [32]:
# Function to explore database tables and contents
def explore_database(conn, table_name=None, limit=5):
    """View database tables and their contents

    Args:
        conn: SQLite connection
        table_name (str, optional): Specific table to explore. If None, list all tables.
        limit (int, optional): Maximum number of rows to display per table
    """
    cursor = conn.cursor()

    # Get list of tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]

    if table_name is None:
        # List all tables and their schema
        print(f"Database contains {len(tables)} tables: {', '.join(tables)}")
        for table in tables:
            print(f"\nTable: {table}")
            cursor.execute(f"PRAGMA table_info({table})")
            columns = cursor.fetchall()
            print("Columns:")
            for col in columns:
                print(f"  {col[1]} ({col[2]}){'  PRIMARY KEY' if col[5] else ''}")
    else:
        # Explore specific table
        if table_name not in tables:
            print(f"Table '{table_name}' not found in database.")
            return

        # Show schema
        print(f"Table: {table_name}")
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        print("Columns:")
        for col in columns:
            print(f"  {col[1]} ({col[2]}){'  PRIMARY KEY' if col[5] else ''}")

        # Show sample data
        cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
        rows = cursor.fetchall()

        if rows:
            print(f"\nSample data ({len(rows)} rows):")
            # Get column names
            col_names = [col[1] for col in columns]

            # Create a DataFrame for nicer display
            df = pd.DataFrame(rows, columns=col_names)
            print(df)
        else:
            print("\nNo data in table.")

In [33]:
# Explore our database tables
print("Database overview:")
explore_database(conn)

Database overview:
Database contains 4 tables: regions, suppliers, products, sales

Table: regions
Columns:
  region_id (INTEGER)  PRIMARY KEY
  name (TEXT)
  risk_factor (INTEGER)

Table: suppliers
Columns:
  supplier_id (INTEGER)  PRIMARY KEY
  name (TEXT)
  country (TEXT)
  region_id (INTEGER)
  tariff_rate (REAL)
  contact_email (TEXT)

Table: products
Columns:
  product_id (INTEGER)  PRIMARY KEY
  name (TEXT)
  category (TEXT)
  price (REAL)
  supplier_id (INTEGER)
  is_purchased (INTEGER)

Table: sales
Columns:
  sale_id (INTEGER)  PRIMARY KEY
  product_id (INTEGER)
  quantity (INTEGER)
  sale_date (TEXT)
  customer_name (TEXT)
  total_amount (REAL)


In [34]:
# Explore specific tables
print("Suppliers table:")
explore_database(conn, "suppliers")

print("\nProducts table:")
explore_database(conn, "products")

print("\nSales table:")
explore_database(conn, "sales")

Suppliers table:
Table: suppliers
Columns:
  supplier_id (INTEGER)  PRIMARY KEY
  name (TEXT)
  country (TEXT)
  region_id (INTEGER)
  tariff_rate (REAL)
  contact_email (TEXT)

Sample data (5 rows):
   supplier_id                 name  country  region_id  tariff_rate  \
0            1   Quantum Microchips      USA          1          0.0   
1            2   Global Electronics    China          2         25.0   
2            3  European Components  Germany          3          5.0   
3            4      Pacific Devices    Japan          2         10.0   
4            5     Nordic Solutions   Sweden          3          5.0   

              contact_email  
0  contact@quantummicro.com  
1      sales@globalelec.com  
2          info@eurocomp.eu  
3     support@pacificdev.jp  
4        hello@nordicsol.se  

Products table:
Table: products
Columns:
  product_id (INTEGER)  PRIMARY KEY
  name (TEXT)
  category (TEXT)
  price (REAL)
  supplier_id (INTEGER)
  is_purchased (INTEGER)

Sample data 

# Natural Language to SQL Conversion

Now let's create functions to convert natural language questions into SQL queries.
We'll implement two approaches:

1. **Rule-based approach**: Using regex patterns to match common question types
2. **LLM-based approach**: Using language models to generate SQL for complex questions

Let's start with the rule-based approach:

In [35]:
# Function to generate SQL from natural language using rules
def rule_based_sql_generation(question):
    """Generate SQL queries based on patterns in the question

    This function uses rules and pattern matching to convert simple natural language
    questions into SQL queries.

    Args:
        question (str): The natural language question

    Returns:
        dict: Contains the generated SQL, query type, and other metadata
    """
    # Normalize the question
    question_lower = question.lower().strip()
    if question_lower.endswith('?'):
        question_lower = question_lower[:-1]

    # Initialize result
    result = {
        "sql": "",
        "query_type": "unknown",
        "confidence": 0.0,
        "matched_pattern": None
    }

    # Pattern 1: Sales from a specific supplier in a specific month
    pattern1 = r"how many sales (did we have |were there |)from (.*?) in ([a-z]+) (\d{4})\??"
    match = re.search(pattern1, question_lower)
    if match:
        supplier_name = match.group(2)
        month_name = match.group(3).capitalize()
        year = match.group(4)

        # Convert month name to number
        month_names = ["January", "February", "March", "April", "May", "June",
                      "July", "August", "September", "October", "November", "December"]
        month_num = month_names.index(month_name) + 1

        sql = f"""
        SELECT COUNT(*) as total_sales, SUM(s.total_amount) as revenue
        FROM sales s
        JOIN products p ON s.product_id = p.product_id
        JOIN suppliers sup ON p.supplier_id = sup.supplier_id
        WHERE sup.name LIKE '%{supplier_name}%'
        AND s.sale_date LIKE '{year}-{month_num:02d}-%'
        """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.9
        result["matched_pattern"] = "supplier_sales_by_month"
        return result

    # Pattern 2: Top N products by revenue/sales
    pattern2 = r"(what are|show me|list) (the |our |)top (\d+) products by (revenue|sales|amount)"
    match = re.search(pattern2, question_lower)
    if match:
        top_n = match.group(3)
        metric = match.group(4)

        if metric in ["revenue", "amount"]:
            sql = f"""
            SELECT p.name, SUM(s.total_amount) as total_revenue
            FROM sales s
            JOIN products p ON s.product_id = p.product_id
            GROUP BY p.product_id
            ORDER BY total_revenue DESC
            LIMIT {top_n}
            """
        else:  # sales count
            sql = f"""
            SELECT p.name, COUNT(*) as total_sales
            FROM sales s
            JOIN products p ON s.product_id = p.product_id
            GROUP BY p.product_id
            ORDER BY total_sales DESC
            LIMIT {top_n}
            """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.85
        result["matched_pattern"] = "top_n_products"
        return result

    # Pattern 3: Products from a specific supplier
    pattern3 = r"(what|which|show|list) products (do we have |are |)(from|by|supplied by) (.*?)(\?|$)"
    match = re.search(pattern3, question_lower)
    if match:
        supplier_name = match.group(4).strip()

        sql = f"""
        SELECT p.name, p.category, p.price
        FROM products p
        JOIN suppliers s ON p.supplier_id = s.supplier_id
        WHERE s.name LIKE '%{supplier_name}%'
        ORDER BY p.category, p.name
        """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.8
        result["matched_pattern"] = "products_by_supplier"
        return result

    # Pattern 4: Sales/revenue by product category
    pattern4 = r"(what is|show|get) (the |)(total |)(sales|revenue) by (product |)category"
    match = re.search(pattern4, question_lower)
    if match:
        metric = match.group(4)

        if metric == "revenue":
            sql = """
            SELECT p.category, SUM(s.total_amount) as total_revenue
            FROM sales s
            JOIN products p ON s.product_id = p.product_id
            GROUP BY p.category
            ORDER BY total_revenue DESC
            """
        else:  # sales count
            sql = """
            SELECT p.category, COUNT(*) as total_sales
            FROM sales s
            JOIN products p ON s.product_id = p.product_id
            GROUP BY p.category
            ORDER BY total_sales DESC
            """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.85
        result["matched_pattern"] = "sales_by_category"
        return result

    # Pattern 5: Sales/revenue in a specific quarter
    pattern5 = r"(what is|show|get) (the |)(total |)(sales|revenue) (in|for) q([1-4]) (\d{4})"
    match = re.search(pattern5, question_lower)
    if match:
        metric = match.group(4)
        quarter = int(match.group(6))
        year = match.group(7)

        # Define quarter date ranges
        quarter_start_month = (quarter - 1) * 3 + 1
        quarter_end_month = quarter * 3

        if metric == "revenue":
            sql = f"""
            SELECT SUM(total_amount) as total_revenue
            FROM sales
            WHERE sale_date >= '{year}-{quarter_start_month:02d}-01'
            AND sale_date <= '{year}-{quarter_end_month:02d}-31'
            """
        else:  # sales count
            sql = f"""
            SELECT COUNT(*) as total_sales
            FROM sales
            WHERE sale_date >= '{year}-{quarter_start_month:02d}-01'
            AND sale_date <= '{year}-{quarter_end_month:02d}-31'
            """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.85
        result["matched_pattern"] = "sales_by_quarter"
        return result

    # Pattern 6: Products sourced locally vs globally
    pattern6 = r"(what|which|show|list) products (are |do we have |)(sourced|purchased) (locally|globally|from local suppliers|from global suppliers)"
    match = re.search(pattern6, question_lower)
    if match:
        source_type = match.group(4)
        is_local = "locally" in source_type or "local" in source_type

        # For local products, we'll consider North America as local (region_id = 1)
        if is_local:
            sql = """
            SELECT p.name, p.category, s.name as supplier, s.country
            FROM products p
            JOIN suppliers s ON p.supplier_id = s.supplier_id
            WHERE p.is_purchased = 1 AND s.region_id = 1
            ORDER BY p.category, p.name
            """
        else:  # global products (non-North America)
            sql = """
            SELECT p.name, p.category, s.name as supplier, s.country, r.name as region
            FROM products p
            JOIN suppliers s ON p.supplier_id = s.supplier_id
            JOIN regions r ON s.region_id = r.region_id
            WHERE p.is_purchased = 1 AND s.region_id != 1
            ORDER BY r.name, p.category, p.name
            """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.9
        result["matched_pattern"] = "products_by_sourcing_location"
        return result

    # Pattern 7: Suppliers with highest risk
    pattern7 = r"(what|which|show|list) suppliers (have|has) (the |)(highest|most|greatest) (risk|geopolitical risk|supply chain risk)"
    match = re.search(pattern7, question_lower)
    if match:
        sql = """
        SELECT s.name, s.country, r.name as region, r.risk_factor
        FROM suppliers s
        JOIN regions r ON s.region_id = r.region_id
        ORDER BY r.risk_factor DESC
        LIMIT 5
        """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.9
        result["matched_pattern"] = "suppliers_by_risk"
        return result

    # Pattern 8: Alternative suppliers in a specific region
    pattern8 = r"(what|which|show|list) (suppliers|alternative suppliers) (do we have |are there |)(in|from) (.*?)( region| area|$)"
    match = re.search(pattern8, question_lower)
    if match:
        region_name = match.group(5).strip()

        sql = f"""
        SELECT s.name, s.country, p.category, COUNT(p.product_id) as product_count
        FROM suppliers s
        JOIN regions r ON s.region_id = r.region_id
        JOIN products p ON s.supplier_id = p.supplier_id
        WHERE r.name LIKE '%{region_name}%'
        GROUP BY s.supplier_id
        ORDER BY product_count DESC
        """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.85
        result["matched_pattern"] = "suppliers_by_region"
        return result

    # Pattern 9: Products affected by tariffs
    pattern9 = r"(what|which|show|list) products (are |)(affected by|subject to|impacted by) (tariffs|high tariffs|tariff rates)"
    match = re.search(pattern9, question_lower)
    if match:
        # Consider tariff rates above 10% as high
        sql = """
        SELECT p.name, p.category, s.name as supplier, s.country, s.tariff_rate
        FROM products p
        JOIN suppliers s ON p.supplier_id = s.supplier_id
        WHERE p.is_purchased = 1 AND s.tariff_rate > 10.0
        ORDER BY s.tariff_rate DESC, p.name
        """

        result["sql"] = sql
        result["query_type"] = "rule_based"
        result["confidence"] = 0.9
        result["matched_pattern"] = "products_with_tariffs"
        return result

    # Default: If no pattern matches, return a simple summary query with low confidence
    sql = """
    SELECT
        (SELECT COUNT(*) FROM suppliers) as supplier_count,
        (SELECT COUNT(*) FROM products) as product_count,
        (SELECT COUNT(*) FROM sales) as sales_count,
        (SELECT SUM(total_amount) FROM sales) as total_revenue
    """

    result["sql"] = sql
    result["query_type"] = "rule_based_default"
    result["confidence"] = 0.3
    result["matched_pattern"] = "default_summary"
    return result

In [36]:
# Test our rule-based SQL generation with a few examples
test_questions = [
    "How many sales did we have from Quantum Microchips in January 2025?",
    "What are our top 3 products by revenue?",
    "Which products are supplied by Nordic Solutions?",
    "What is the total revenue by product category?",
    "What is the total sales for Q1 2025?",
    "Which products are sourced locally?",
    "What suppliers have the highest geopolitical risk?",
    "What alternative suppliers do we have in Asia region?",
    "Which products are affected by high tariffs?"
]

for question in test_questions:
    print(f"Question: {question}")
    result = rule_based_sql_generation(question)
    print(f"Matched pattern: {result['matched_pattern']} (Confidence: {result['confidence']})")
    print(f"SQL: {result['sql']}")
    print("-" * 80)

Question: How many sales did we have from Quantum Microchips in January 2025?
Matched pattern: supplier_sales_by_month (Confidence: 0.9)
SQL: 
        SELECT COUNT(*) as total_sales, SUM(s.total_amount) as revenue
        FROM sales s
        JOIN products p ON s.product_id = p.product_id
        JOIN suppliers sup ON p.supplier_id = sup.supplier_id
        WHERE sup.name LIKE '%quantum microchips%'
        AND s.sale_date LIKE '2025-01-%'
        
--------------------------------------------------------------------------------
Question: What are our top 3 products by revenue?
Matched pattern: top_n_products (Confidence: 0.85)
SQL: 
            SELECT p.name, SUM(s.total_amount) as total_revenue
            FROM sales s
            JOIN products p ON s.product_id = p.product_id
            GROUP BY p.product_id
            ORDER BY total_revenue DESC
            LIMIT 3
            
--------------------------------------------------------------------------------
Question: Which product

Now let's implement the LLM-based approach for more complex questions:

In [37]:
# Function to generate SQL using an LLM
def generate_sql_with_llm(question, api_key=None, model="openai"):
    """Generate SQL using an LLM

    Args:
        question (str): The natural language question
        api_key (str): API key for the LLM service
        model (str): Which model to use - "openai" or "deepseek"

    Returns:
        dict: Contains the generated SQL and metadata
    """
    # If no API key is provided, return an empty result
    if not api_key:
        return {
            "sql": "",
            "query_type": "llm_failed",
            "confidence": 0.0,
            "error": "No API key provided"
        }

    # Get database schema for context
    schema_context = """
    Database Schema:

    regions (region_id, name, risk_factor)
    suppliers (supplier_id, name, country, contact_email, region_id, tariff_rate)
    products (product_id, name, category, price, supplier_id, is_purchased)
    sales (sale_id, product_id, quantity, sale_date, customer_name, total_amount)

    The regions table contains information about geographical regions and their geopolitical risk factors (1-10 scale).
    The suppliers table contains information about product suppliers, including their region and tariff rates.
    The products table contains product details and references suppliers. The is_purchased flag (1 or 0) indicates if we currently purchase this product.
    The sales table contains sales transactions and references products (only for products with is_purchased=1).
    """

    # Prompt for SQL generation
    prompt = f"""
    {schema_context}

    This database focuses on supply chain management in the context of geopolitical fragmentation.
    Key concepts include:
    - Regions have risk factors representing geopolitical stability (1-10 scale)
    - Suppliers are associated with regions and have tariff rates
    - Products can be either currently purchased (is_purchased=1) or potential alternatives (is_purchased=0)
    - Sales data only exists for currently purchased products

    Based on the schema above, write a SQL query to answer this question: "{question}"

    Return ONLY the SQL query without any explanations or markdown formatting.
    Make sure the query is correct and executable in SQLite.
    Join tables where necessary to get the required information.
    """

    try:
        # Choose the appropriate API endpoint
        if model == "openai":
            API_URL = "https://api.openai.com/v1/chat/completions"
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {api_key}"
            }
            payload = {
                "model": "gpt-3.5-turbo",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that converts natural language questions into SQL queries."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": 0.3
            }
        elif model == "deepseek":
            API_URL = "https://api.deepseek.com/v1/chat/completions"
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {api_key}"
            }
            payload = {
                "model": "deepseek-chat",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that converts natural language questions into SQL queries."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": 0.3
            }
        else:
            return {
                "sql": "",
                "query_type": "llm_failed",
                "confidence": 0.0,
                "error": f"Unsupported model: {model}"
            }

        # Make the API request
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()

        # Parse the response
        result = response.json()
        sql = result["choices"][0]["message"]["content"].strip()

        # Clean up the SQL (remove markdown code blocks if present)
        if sql.startswith("```sql"):
            sql = sql.split("```sql", 1)[1]
        if sql.startswith("```"):
            sql = sql.split("```", 1)[1]
        if sql.endswith("```"):
            sql = sql.rsplit("```", 1)[0]

        sql = sql.strip()

        return {
            "sql": sql,
            "query_type": f"llm_{model}",
            "confidence": 0.8,
            "model": model
        }

    except Exception as e:
        return {
            "sql": "",
            "query_type": "llm_failed",
            "confidence": 0.0,
            "error": str(e)
        }
# Function to get OpenAI API key
def get_openai_api_key():
    """Get the OpenAI API key from environment variable or user input"""
    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        # Try to get from user input
        try:
            from getpass import getpass
            api_key = getpass("Enter your OpenAI API key: ")
            os.environ["OPENAI_API_KEY"] = api_key
        except Exception as e:
            print(f"Error getting API key: {e}")
            return None
    return api_key

In [38]:
# Function to get Deepseek API key
def get_deepseek_api_key():
    """Get the Deepseek API key from environment variable or user input"""
    api_key = os.environ.get("DEEPSEEK_API_KEY")
    if not api_key:
        # Try to get from user input
        try:
            from getpass import getpass
            api_key = getpass("Enter your Deepseek API key: ")
            os.environ["DEEPSEEK_API_KEY"] = api_key
        except Exception as e:
            print(f"Error getting API key: {e}")
            return None
    return api_key

## Combined SQL Generation Approach

Now let's create a function that combines both approaches:
1. First try the rule-based approach
2. If confidence is low, fall back to the LLM approach if an API key is available
3. If LLM approach fails or is disabled, use the rule-based result

In [39]:
def generate_sql_query(question, use_llm=True):
    """Generate SQL from natural language using a combined approach

    Args:
        question (str): The natural language question
        use_llm (bool): Whether to use LLM as fallback for low-confidence rule matches

    Returns:
        dict: Contains the generated SQL and metadata
    """
    # First try rule-based approach
    rule_result = rule_based_sql_generation(question)

    # If rule-based approach has high confidence, use it
    if rule_result["confidence"] >= 0.7:
        return rule_result

    # Otherwise, try LLM-based approach if enabled
    if use_llm:
        # Try OpenAI first
        api_key = get_openai_api_key()
        if api_key:
            llm_result = generate_sql_with_llm(question, api_key, "openai")
            if llm_result["sql"]:
                return llm_result

        # Try Deepseek as fallback
        api_key = get_deepseek_api_key()
        if api_key:
            llm_result = generate_sql_with_llm(question, api_key, "deepseek")
            if llm_result["sql"]:
                return llm_result

    # If LLM approach fails or is disabled, return the rule-based result anyway
    return rule_result

In [40]:
# Test our combined SQL generation approach
complex_questions = [
    "How many sales did we have from Quantum Microchips in January 2025?",
    "What is the average price of products in each category?",
    "Which supplier provides the most products in the Electronics category?",
    "What was the total revenue from each supplier in Q1 2025?",
    "Who are our top 3 customers by total purchase amount?"
]

for question in complex_questions:
    print(f"Question: {question}")

    # Try without LLM first
    print("Rule-based approach:")
    rule_result = rule_based_sql_generation(question)
    print(f"  Confidence: {rule_result['confidence']}")
    print(f"  SQL: {rule_result['sql'][:100]}...")

    # Try combined approach (with LLM if available)
    print("Combined approach:")
    result = generate_sql_query(question, use_llm=False)  # Set to True to use LLM if available
    print(f"  Method: {result['query_type']}")
    print(f"  Confidence: {result['confidence']}")
    print(f"  SQL: {result['sql'][:100]}...")
    print("-" * 80)

Question: How many sales did we have from Quantum Microchips in January 2025?
Rule-based approach:
  Confidence: 0.9
  SQL: 
        SELECT COUNT(*) as total_sales, SUM(s.total_amount) as revenue
        FROM sales s
       ...
Combined approach:
  Method: rule_based
  Confidence: 0.9
  SQL: 
        SELECT COUNT(*) as total_sales, SUM(s.total_amount) as revenue
        FROM sales s
       ...
--------------------------------------------------------------------------------
Question: What is the average price of products in each category?
Rule-based approach:
  Confidence: 0.3
  SQL: 
    SELECT
        (SELECT COUNT(*) FROM suppliers) as supplier_count,
        (SELECT COUNT(*) FRO...
Combined approach:
  Method: rule_based_default
  Confidence: 0.3
  SQL: 
    SELECT
        (SELECT COUNT(*) FROM suppliers) as supplier_count,
        (SELECT COUNT(*) FRO...
--------------------------------------------------------------------------------
Question: Which supplier provides the most produ

# Query Execution and Result Formatting

Now that we can generate SQL queries from natural language questions, let's create functions to execute these queries and format the results in a way that's useful for the LLM.

In [41]:
def execute_sql_query(conn, sql):
    """Execute a SQL query and return the results

    Args:
        conn: SQLite connection
        sql (str): SQL query to execute

    Returns:
        dict: Contains query results, success status, and error message if any
    """
    result = {
        "success": False,
        "data": None,
        "error": None,
        "column_names": [],
        "row_count": 0
    }

    try:
        # Execute the query
        df = pd.read_sql_query(sql, conn)

        # Store the results
        result["success"] = True
        result["data"] = df
        result["column_names"] = df.columns.tolist()
        result["row_count"] = len(df)

    except Exception as e:
        result["error"] = str(e)

    return result

In [42]:
def format_query_results(query_result):
    """Format query results for display and LLM context

    Args:
        query_result (dict): Result from execute_sql_query

    Returns:
        str: Formatted string representation of the results
    """
    if not query_result["success"]:
        return f"Error executing query: {query_result['error']}"

    df = query_result["data"]

    if query_result["row_count"] == 0:
        return "The query returned no results."

    # Format as markdown table for better display
    result_str = f"Query returned {query_result['row_count']} rows with columns: {', '.join(query_result['column_names'])}\n\n"

    # Add table header
    result_str += "| " + " | ".join(query_result["column_names"]) + " |\n"
    result_str += "| " + " | ".join(["---" for _ in query_result["column_names"]]) + " |\n"

    # Add table rows (limit to 20 rows for readability)
    max_rows = min(20, query_result["row_count"])
    for i in range(max_rows):
        row = df.iloc[i]
        result_str += "| " + " | ".join([str(row[col]) for col in query_result["column_names"]]) + " |\n"

    # Add summary if there are more rows
    if query_result["row_count"] > max_rows:
        result_str += f"\n... and {query_result['row_count'] - max_rows} more rows (showing first {max_rows} only)"

    # Add some basic statistics for numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        result_str += "\n\nSummary statistics for numeric columns:\n\n"
        for col in numeric_cols:
            result_str += f"- {col}: min={df[col].min():.2f}, max={df[col].max():.2f}, avg={df[col].mean():.2f}\n"

    return result_str

In [43]:
# Test our query execution and formatting
test_queries = [
    "SELECT * FROM suppliers LIMIT 3",
    "SELECT p.name, p.price, s.name as supplier FROM products p JOIN suppliers s ON p.supplier_id = s.supplier_id LIMIT 5",
    "SELECT COUNT(*) as total_sales, SUM(total_amount) as revenue FROM sales WHERE sale_date LIKE '2025-01-%'"
]

for sql in test_queries:
    print(f"Executing SQL: {sql}")
    result = execute_sql_query(conn, sql)
    formatted = format_query_results(result)
    print("\nFormatted Results:")
    print(formatted)
    print("-" * 80)

Executing SQL: SELECT * FROM suppliers LIMIT 3

Formatted Results:
Query returned 3 rows with columns: supplier_id, name, country, region_id, tariff_rate, contact_email

| supplier_id | name | country | region_id | tariff_rate | contact_email |
| --- | --- | --- | --- | --- | --- |
| 1 | Quantum Microchips | USA | 1 | 0.0 | contact@quantummicro.com |
| 2 | Global Electronics | China | 2 | 25.0 | sales@globalelec.com |
| 3 | European Components | Germany | 3 | 5.0 | info@eurocomp.eu |


Summary statistics for numeric columns:

- supplier_id: min=1.00, max=3.00, avg=2.00
- region_id: min=1.00, max=3.00, avg=2.00
- tariff_rate: min=0.00, max=25.00, avg=10.00

--------------------------------------------------------------------------------
Executing SQL: SELECT p.name, p.price, s.name as supplier FROM products p JOIN suppliers s ON p.supplier_id = s.supplier_id LIMIT 5

Formatted Results:
Query returned 5 rows with columns: name, price, supplier

| name | price | supplier |
| --- | --- | -

# LLM Integration for Answer Generation

Now let's implement the answer generation part of our RAG pipeline. We'll provide three options:

1. **OpenAI API**: Using GPT models for high-quality answers
2. **Deepseek API**: As an alternative commercial API
3. **Hugging Face API**: Using free models like google/flan-t5-base
4. **Simple Extractive Approach**: A free option that doesn't require API keys

In [44]:
def generate_answer_with_llm(question, context, api_key=None, model="openai"):
    """Generate an answer using an LLM

    Args:
        question (str): The natural language question
        context (str): The context from SQL query results
        api_key (str): API key for the LLM service
        model (str): Which model to use - "openai" or "deepseek"

    Returns:
        str: The generated answer
    """
    # If no API key is provided, return None
    if not api_key:
        return "No API key provided for LLM answer generation."

    # Prompt for answer generation
    prompt = f"""
    You are an AI assistant that answers questions about business data.

    Question: {question}

    Here are the database query results to help you answer this question:

    {context}

    Please provide a clear, concise answer to the question based on the data provided.
    Include relevant numbers and insights, but keep your response focused and to the point.
    If the data doesn't contain enough information to answer the question, say so.
    """

    try:
        # Choose the appropriate API endpoint
        if model == "openai":
            API_URL = "https://api.openai.com/v1/chat/completions"
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {api_key}"
            }
            payload = {
                "model": "gpt-3.5-turbo",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that answers questions about business data."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": 0.3
            }
        elif model == "deepseek":
            API_URL = "https://api.deepseek.com/v1/chat/completions"
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {api_key}"
            }
            payload = {
                "model": "deepseek-chat",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that answers questions about business data."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "temperature": 0.3
            }
        else:
            return f"Unsupported model: {model}"

        # Make the API request
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()

        # Parse the response
        result = response.json()
        answer = result["choices"][0]["message"]["content"].strip()

        return answer

    except Exception as e:
        return f"Error generating answer with LLM: {str(e)}"

In [45]:
# Generate answer using Hugging Face API (free option)
def generate_answer_with_huggingface(question, context, api_key=None):
    """Generate an answer using Hugging Face API

    Args:
        question (str): The natural language question
        context (str): The context from SQL query results
        api_key (str): Hugging Face API token

    Returns:
        str: The generated answer
    """
    if not api_key:
        return "No Hugging Face API token provided."

    API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-base"
    headers = {"Authorization": f"Bearer {api_key}"}

    # Create a prompt for the model
    prompt = f"""
    Question: {question}

    Context:
    {context}

    Answer:
    """

    try:
        response = requests.post(API_URL, headers=headers, json={"inputs": prompt})
        response.raise_for_status()

        # Parse the response
        result = response.json()
        if isinstance(result, list):
            answer = result[0]["generated_text"]
        else:
            answer = result["generated_text"]

        return answer

    except Exception as e:
        return f"Error generating answer with Hugging Face: {str(e)}"

In [46]:
# Function to get Hugging Face API key
def get_huggingface_api_key():
    """Get the Hugging Face API key from environment variable or user input"""
    api_key = os.environ.get("HUGGINGFACE_API_KEY")
    if not api_key:
        # Try to get from user input
        try:
            from getpass import getpass
            api_key = getpass("Enter your Hugging Face API token: ")
            os.environ["HUGGINGFACE_API_KEY"] = api_key
        except Exception as e:
            print(f"Error getting API key: {e}")
            return None
    return api_key

In [47]:
# Simple extractive answer generation (no API key required)
def simple_extractive_answer(question, query_result):
    """Generate a simple answer from query results without using an LLM

    Args:
        question (str): The natural language question
        query_result (dict): The query execution result

    Returns:
        str: A simple answer based on the query results
    """
    if not query_result["success"]:
        return f"Sorry, I couldn't answer that question due to an error: {query_result['error']}"

    df = query_result["data"]

    if query_result["row_count"] == 0:
        return "I couldn't find any data to answer your question."

    # Prepare a simple answer based on the query type and results
    question_lower = question.lower()

    # For geopolitical risk questions
    if any(word in question_lower for word in ["risk", "geopolitical", "risky"]):
        if "risk_factor" in df.columns:
            # Sort by risk_factor if available
            if df.shape[0] > 0:
                suppliers = []
                for i in range(min(5, df.shape[0])):
                    row = df.iloc[i]
                    if "name" in df.columns and "risk_factor" in df.columns:
                        name_col = df.columns.get_loc("name")
                        risk_col = df.columns.get_loc("risk_factor")
                        suppliers.append(f"{row[name_col]} (risk: {row[risk_col]})")
                    else:
                        suppliers.append(f"{row[0]} (risk: {row[1]})")
                return f"The highest risk suppliers are: {', '.join(suppliers)}"

    # For local vs global sourcing questions
    if any(phrase in question_lower for phrase in ["local", "global", "sourced", "region"]):
        if df.shape[0] > 0:
            products = []
            for i in range(min(5, df.shape[0])):
                row = df.iloc[i]
                if "name" in df.columns:
                    name_col = df.columns.get_loc("name")
                    products.append(f"{row[name_col]}")
                else:
                    products.append(f"{row[0]}")

            if "local" in question_lower:
                return f"Products sourced locally: {', '.join(products)}"
            elif "global" in question_lower:
                return f"Products sourced globally: {', '.join(products)}"
            else:
                return f"Products in this region: {', '.join(products)}"

    # For tariff-related questions
    if any(word in question_lower for word in ["tariff", "tariffs", "tax", "taxes"]):
        if df.shape[0] > 0:
            items = []
            for i in range(min(5, df.shape[0])):
                row = df.iloc[i]
                if "name" in df.columns and "tariff_rate" in df.columns:
                    name_col = df.columns.get_loc("name")
                    tariff_col = df.columns.get_loc("tariff_rate")
                    items.append(f"{row[name_col]} ({row[tariff_col]}%)")
                else:
                    items.append(f"{row[0]}")
            return f"Products/suppliers affected by tariffs: {', '.join(items)}"

    # For questions about totals or counts
    if any(word in question_lower for word in ["how many", "count", "total"]):
        if "count" in df.columns or "total" in df.columns:
            col = "count" if "count" in df.columns else "total"
            value = df[col].sum()
            return f"The total is {value}."
        elif df.shape[0] == 1 and df.shape[1] == 1:
            # If result is a single value
            value = df.iloc[0, 0]
            return f"The result is {value}."

    # For questions about top items
    if "top" in question_lower and df.shape[0] > 0:
        if df.shape[0] == 1:
            row = df.iloc[0]
            return f"The top item is {row[0]} with a value of {row[1]}."
        else:
            items = []
            for i in range(min(3, df.shape[0])):
                row = df.iloc[i]
                items.append(f"{row[0]} ({row[1]})")
            return f"The top items are: {', '.join(items)}."

    # For questions about averages
    if any(word in question_lower for word in ["average", "mean"]):
        for col in df.select_dtypes(include=['number']).columns:
            if "price" in col.lower() or "amount" in col.lower() or "value" in col.lower():
                avg_value = df[col].mean()
                return f"The average {col} is {avg_value:.2f}."

    # Default: return a summary of the results
    if df.shape[0] == 1:
        # Single row result
        row = df.iloc[0]
        cols = df.columns
        return f"I found: {', '.join([f'{cols[i]}: {row[i]}' for i in range(len(cols))])}"
    else:
        # Multiple row result
        return f"I found {df.shape[0]} results. First few items: {', '.join([str(df.iloc[i, 0]) for i in range(min(3, df.shape[0]))])}"

# Complete Database RAG Pipeline

Now let's put everything together to create our complete database RAG pipeline:
1. Take a natural language question
2. Convert it to SQL
3. Execute the SQL query
4. Format the results
5. Generate an answer using an LLM (or simple extractive approach)

In [48]:
def db_rag_pipeline(question, conn, sql_method="combined", answer_method="extractive"):
    """Complete database RAG pipeline

    Args:
        question (str): Natural language question
        conn: SQLite connection
        sql_method (str): Method for SQL generation - "rule", "llm", or "combined"
        answer_method (str): Method for answer generation - "openai", "deepseek", "huggingface", or "extractive"

    Returns:
        dict: Contains all intermediate results and the final answer
    """
    result = {
        "question": question,
        "sql_generation": None,
        "query_execution": None,
        "formatted_results": None,
        "answer": None,
        "error": None
    }

    try:
        # Step 1: Generate SQL from the question
        if sql_method == "rule":
            sql_result = rule_based_sql_generation(question)
        elif sql_method == "llm":
            # Try OpenAI first, then Deepseek
            api_key = get_openai_api_key()
            if api_key:
                sql_result = generate_sql_with_llm(question, api_key, "openai")
            else:
                api_key = get_deepseek_api_key()
                if api_key:
                    sql_result = generate_sql_with_llm(question, api_key, "deepseek")
                else:
                    sql_result = {
                        "sql": "",
                        "query_type": "llm_failed",
                        "confidence": 0.0,
                        "error": "No API keys available for LLM SQL generation"
                    }
        else:  # combined
            sql_result = generate_sql_query(question, use_llm=True)

        result["sql_generation"] = sql_result

        # Check if SQL generation was successful
        if not sql_result["sql"]:
            result["error"] = f"SQL generation failed: {sql_result.get('error', 'Unknown error')}"
            return result

        # Step 2: Execute the SQL query
        query_result = execute_sql_query(conn, sql_result["sql"])
        result["query_execution"] = query_result

        # Check if query execution was successful
        if not query_result["success"]:
            result["error"] = f"Query execution failed: {query_result['error']}"
            return result

        # Step 3: Format the results
        formatted_results = format_query_results(query_result)
        result["formatted_results"] = formatted_results

        # Step 4: Generate an answer
        if answer_method == "openai":
            api_key = get_openai_api_key()
            if api_key:
                answer = generate_answer_with_llm(question, formatted_results, api_key, "openai")
            else:
                answer = "OpenAI API key not available. Please try another answer method."

        elif answer_method == "deepseek":
            api_key = get_deepseek_api_key()
            if api_key:
                answer = generate_answer_with_llm(question, formatted_results, api_key, "deepseek")
            else:
                answer = "Deepseek API key not available. Please try another answer method."

        elif answer_method == "huggingface":
            api_key = get_huggingface_api_key()
            if api_key:
                answer = generate_answer_with_huggingface(question, formatted_results, api_key)
            else:
                answer = "Hugging Face API token not available. Please try another answer method."

        else:  # extractive
            answer = simple_extractive_answer(question, query_result)

        result["answer"] = answer

    except Exception as e:
        result["error"] = f"Pipeline error: {str(e)}"

    return result

In [49]:
# Test the complete pipeline with example questions
test_questions = [
    "How many products do we have in each category?",
    "What are the top 3 selling products in Q1 2025?",
    "Which supplier has the highest total sales in January 2025?"
]

for question in test_questions:
    print(f"Question: {question}")

    # Use the rule-based approach and extractive answer generation for testing
    result = db_rag_pipeline(question, conn, sql_method="rule", answer_method="extractive")

    print(f"SQL: {result['sql_generation']['sql']}")
    print("\nQuery Results:")
    print(result["formatted_results"])
    print("\nAnswer:")
    print(result["answer"])
    print("-" * 80)

Question: How many products do we have in each category?
SQL: 
    SELECT
        (SELECT COUNT(*) FROM suppliers) as supplier_count,
        (SELECT COUNT(*) FROM products) as product_count,
        (SELECT COUNT(*) FROM sales) as sales_count,
        (SELECT SUM(total_amount) FROM sales) as total_revenue
    

Query Results:
Query returned 1 rows with columns: supplier_count, product_count, sales_count, total_revenue

| supplier_count | product_count | sales_count | total_revenue |
| --- | --- | --- | --- |
| 9.0 | 23.0 | 257.0 | 441931.19999999984 |


Summary statistics for numeric columns:

- supplier_count: min=9.00, max=9.00, avg=9.00
- product_count: min=23.00, max=23.00, avg=23.00
- sales_count: min=257.00, max=257.00, avg=257.00
- total_revenue: min=441931.20, max=441931.20, avg=441931.20


Answer:
I found: supplier_count: 9.0, product_count: 23.0, sales_count: 257.0, total_revenue: 441931.19999999984
----------------------------------------------------------------------------

/tmp/ipython-input-2080723802.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return f"I found: {', '.join([f'{cols[i]}: {row[i]}' for i in range(len(cols))])}"
/tmp/ipython-input-2080723802.py:87: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return f"The top item is {row[0]} with a value of {row[1]}."
/tmp/ipython-input-2080723802.py:107: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return f"I found: {', '.join([f'{cols[i]}

# Interactive UI for Testing

Let's create an interactive UI using Jupyter widgets to test our database RAG system:

In [50]:
# Import ipywidgets for interactive UI
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [51]:
# Create the interactive UI
def create_interactive_ui():
    """Create an interactive UI for testing the database RAG system"""
    # Create input widgets
    question_input = widgets.Text(
        value="What are the top 3 selling products in Q1 2025?",
        placeholder="Enter your question here",
        description="Question:",
        disabled=False,
        style={"description_width": "100px"},
        layout=widgets.Layout(width="80%")
    )

    sql_method_dropdown = widgets.Dropdown(
        options=["rule", "llm", "combined"],
        value="rule",
        description="SQL Method:",
        disabled=False,
        style={"description_width": "100px"}
    )

    answer_method_dropdown = widgets.Dropdown(
        options=["extractive", "openai", "deepseek", "huggingface"],
        value="extractive",
        description="Answer Method:",
        disabled=False,
        style={"description_width": "100px"}
    )

    submit_button = widgets.Button(
        description="Submit",
        button_style="primary",
        tooltip="Submit your question",
        icon="search"
    )

    # Create output widgets
    output_area = widgets.Output()

    # Define button click handler
    def on_submit_button_clicked(b):
        with output_area:
            clear_output()
            print(f"Processing question: {question_input.value}")
            print(f"SQL Method: {sql_method_dropdown.value}")
            print(f"Answer Method: {answer_method_dropdown.value}")
            print("-" * 80)

            try:
                # Make sure we have a valid database connection
                global conn
                if 'conn' not in globals() or conn is None:
                    # Create a new connection if needed
                    print("Creating new database connection...")
                    import sqlite3
                    conn = sqlite3.connect("supply_chain.db")

                # Run the pipeline
                result = db_rag_pipeline(
                    question_input.value,
                    conn,
                    sql_method=sql_method_dropdown.value,
                    answer_method=answer_method_dropdown.value
                )

                # Display results
                if "error" in result and result["error"]:
                    print(f"Error: {result['error']}")
                else:
                    print("Generated SQL:")
                    print(f"```sql\n{result['sql_generation']['sql']}\n```")
                    print(f"Method: {result['sql_generation']['query_type']}, Confidence: {result['sql_generation']['confidence']}")
                    print("\nQuery Results:")
                    print(result["formatted_results"])
                    print("\nAnswer:")
                    if "answer" in result and result["answer"]:
                        print(result["answer"])
                    else:
                        print("No answer was generated. Please try a different question or answer method.")
            except Exception as e:
                print(f"Error in UI: {str(e)}")
                import traceback
                traceback.print_exc()

    # Connect the button click event to the handler
    submit_button.on_click(on_submit_button_clicked)

    # Create the layout
    ui = widgets.VBox([
        widgets.HTML("<h2>Database RAG System</h2>"),
        widgets.HTML("<p>Ask questions about suppliers, products, and sales data</p>"),
        question_input,
        widgets.HBox([sql_method_dropdown, answer_method_dropdown]),
        submit_button,
        widgets.HTML("<hr>"),
        output_area
    ])

    return ui

In [52]:
# Display the interactive UI
ui = create_interactive_ui()
display(ui)

# Conclusion

In this tutorial, we've built a complete database RAG (Retrieval Augmented Generation) system that allows users to ask natural language questions about structured data. Our system:

1. **Converts natural language to SQL** using either rule-based patterns or LLM-based generation
2. **Executes SQL queries** against a SQLite database
3. **Formats query results** in a readable way
4. **Generates answers** using either LLMs (OpenAI, Deepseek, Hugging Face) or a simple extractive approach
5. Provides an **interactive UI** for testing

This approach demonstrates how to combine the power of LLMs with structured data querying, providing accurate answers based on your database content.

## Possible Extensions

Here are some ways you could extend this system:

1. **Add more rule patterns** to handle additional question types
2. **Implement SQL validation** to ensure generated queries are safe and correct
3. **Add caching** to avoid regenerating the same SQL queries
4. **Create visualizations** based on query results
5. **Integrate with local LLMs** like Ollama for fully offline operation
6. **Add query history** to track and reuse previous questions and answers
7. **Implement feedback mechanisms** to improve SQL generation over time

Happy querying!